In [1]:
import cv2 as cv
import numpy as np
import math

video = cv.VideoCapture(0)

while video.isOpened():
    
    ret, frame = video.read()
    #print(ret)
    #print(frame.shape)
    
    cv.rectangle(frame, (50,50), (350,350), (255,0,255),2)
    
    cv.circle(frame, (50,50), 10, (255,0,255), -1)
    crop_image = frame[50:350, 50:350]

    #applying gaussian blurr to remove noise
    blur = cv.GaussianBlur(crop_image, (3,3), 0)
    
    # changing color space to hsv
    hsv = cv.cvtColor(blur, cv.COLOR_BGR2HSV)
    
    # filtering out hand skin color by creating a mask
    
    mask_2 = cv.inRange(hsv, np.array([2,0,0]), np.array([20,255,255]))
    
    # removing noise from crop_image by performing dilation and erosion respectively
    kernel = np.ones((5,5))
    dilation = cv.dilate(mask_2, kernel, iterations = 1)
    erosion = cv.erode(dilation, kernel, iterations = 1)
    
    # filtering image
    filtered = cv.GaussianBlur(erosion, (3,3), 0)
    ret, thresh = cv.threshold(filtered, 127, 255, cv.THRESH_BINARY)
    
    # finding contours in our image
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    try:
        # finding contour with maximum area
        contour = max(contours, key=lambda x: cv.contourArea(x))
        
        #creating bounding rectangle around the contour
        x,y,w,h = cv.boundingRect(contour)
        cv.rectangle(crop_image, (x,y), (x+w,y+h),(255,0,0), 1)
        
        # finding convex hull
        hull = cv.convexHull(contour)
        
        # drawing contours
        drawing = np.zeros(crop_image.shape, dtype = "uint8")
        cv.drawContours(drawing, [contour], -1, (0,255,0),0)
        cv.drawContours(drawing, [hull], -1, (0,255,0),0)
        
        # finding convexity defects
        hull = cv.convexHull(contour, returnPoints = False)
        defects = cv.convexityDefects(contour, hull)
        
        # Use cosine rule to find angle of the far point from the start and end point i.e. the convex points (the finger
        # tips) for all defects
        count_defects = 0
        
        for i in range(defects.shape[0]):
            s, e, f, d = defects[i, 0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])
            #print("start point: {}".format(start))
            #print("end point: {}".format(end))
            #print("far point: {}".format(far))
            #print("depth: {}".format(d))
            
            
            
            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14
            
            # if angle > 90 draw a circle at the far point
            if angle <= 90:
                count_defects += 1
                cv.circle(crop_image, far, 1, [0, 255, 255], -1)
            
            cv.line(crop_image, start, end, [0, 255, 0], 2)
        
        # Print number of fingers
        #print("now counting fingers")
        if count_defects == 0:
            cv.putText(frame, "ONE", (50, 50), cv.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2)
        elif count_defects == 1:
            cv.putText(frame, "TWO", (50, 50), cv.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 2:
            cv.putText(frame, "THREE", (5, 50), cv.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 3:
            cv.putText(frame, "FOUR", (50, 50), cv.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 4:
            cv.putText(frame, "FIVE", (50, 50), cv.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        else:
            pass
        
        
    except:
        print("inside except")
        pass
    
    cv.imshow("hand identified", frame)
    cv.imshow("threshold image", thresh)
    #cv.imshow("filtered image", filtered)
    all_image = np.hstack((drawing, crop_image))
    cv.imshow('Contours', all_image)
    
    if cv.waitKey(5) & 0xFF == ord('q'):
        break
        
video.release()
cv.destroyAllWindows()